In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os

In [88]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [89]:
response = requests.get(url)

In [90]:
soup = bs(response.text, 'html.parser')

# Scraping first headline 

In [114]:

results = soup.find_all('div', class_="content_title")[0]

news_title = results.text.strip()
news_title


"NASA's Perseverance Drives on Mars' Terrain for First Time"

# Scraping first teaser paragraph

In [120]:
results = soup.find_all("div", class_= "rollover_description_inner")[0]
news_p = results.text.strip()
news_p

'The first trek of the agency’s largest, most advanced rover yet on the Red Planet marks a major milestone before science operations get under way.'

# code for multiple headlines -->

In [111]:
#print(len(headlines))

#-----------------------------
# headlines = []
# for result in results:
#     if(result.a):
#         if(result.a.text):
#             headlines.append(result)
# print(headlines)

# clean_headlines = []
# for x in range(6):
#     head_II = headlines[x].text.strip()
#     clean_headlines.append(head_II)
    
# clean_headlines

["NASA's Perseverance Drives on Mars' Terrain for First Time",
 'NASA Awards Mars Ascent Propulsion System Contract for Sample Return',
 "NASA to Provide Update on Perseverance ‘Firsts' Since Mars Landing",
 "NASA's Mars Perseverance Rover Provides Front-Row Seat to Landing, First Audio Recording of Red Planet",
 'NASA to Reveal New Video, Images From Mars Perseverance Rover',
 "NASA's Next Mars Rover Is Ready for the Most Precise Landing Yet"]

In [26]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [27]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/charlie/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


# JPL Mars Space Images - Featured Image

In [28]:
url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(url)

In [37]:
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
images = soup.find_all('div', class_='floating_text_area')

for image in images:
    link = image.find("a")
    href = link['href']
    
    print("https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/" + href)
    
featured_image_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'
    

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg
